# Introducing: Feature modifiers

## Basic example

In [6]:
import { sns, kms, nb } from './lib/index.ts';

const stack1 = nb.stack();

const sourceTopic = new sns.L1Topic(stack1, "IncomingMessages");
sns.encrypt(sourceTopic, new kms.Key(stack1, "TopicKey"))
sns.fifo(sourceTopic, {
  contentBasedDeduplication: true,
});

nb.print(stack1);

Resources:
  IncomingMessages:
    Type: 'AWS::SNS::Topic'
    Properties:
      kmsMasterKeyId:
        'Fn::GetAtt':
          - TopicKeyB2E0C9CB
          - Arn
      topicName: ahk66af-IncomingMessages-EF9577EB.fifo
      fifoTopic: true
      contentBasedDeduplication: true
  TopicKeyB2E0C9CB:
    Type: 'AWS::KMS::Key'
    Properties:
      KeyPolicy:
        Statement:
          - Action: 'kms:*'
            Effect: Allow
            Principal:
              AWS:
                'Fn::Join':
                  - ''
                  - - 'arn:'
                    - Ref: 'AWS::Partition'
                    - ':iam::'
                    - Ref: 'AWS::AccountId'
                    - ':root'
            Resource: '*'
        Version: '2012-10-17'
    UpdateReplacePolicy: Retain
    DeletionPolicy: Retain



## Add a Queue as a target for the Topic

### Excursion: Syntax variations

This uses a variation of the modifiers syntax: Instead of directly calling the modifier, we pass it to the `with()` method of the resource construct.s
The downside of it is limited support in code auto-completion. `with()` would accept *any* modifier, so we can't suggest which modifier to use and what parameters the modifier supports. A workaround would be to use overloads in the method definition. With overloads, we get a list of expected modifiers and once the first parameter is provided, the auto-completion appears to be able to pick the right one.

<img src="images/modifiers_with_help.png" width="400" class="blog-image">

Line 8 uses yet another alternative syntax, which improves code auto-completion. These additional methods would have to be codegen'd and would only support built-in modifiers.

<img src="images/modifiers_use_help.png" width="450" class="blog-image">

### Inspired by pipeline operator

The syntax for these is inspired by the *tc39 pipeline operator proposal*. Any variation of it could be considered going forward. Crucially, it is syntactic sugar on top of the base implementation of a function that takes a subject construct as first parameter. See [this article implementing the pipeline operator proposal](https://dev.to/nexxeln/implementing-the-pipe-operator-in-typescript-30ip) for further information.

In [21]:
import { cdk, sqs, nb } from './lib/index.ts';

const stack2 = nb.stack();

const targetQueue = new sqs.L1Queue(stack2, "AcceptedMessages")
  .with(sqs.visibilityTimeout, cdk.Duration.seconds(300))
  .with(sqs.enforceSSL)
  .useEncrypt();

nb.print(stack2);

Resources:
  AcceptedMessages:
    Type: 'AWS::SQS::Queue'
    Properties:
      visibilityTimeout: 300
      kmsMasterKeyId: alias/aws/sqs
  AcceptedMessagesPolicyA013B031:
    Type: 'AWS::SQS::QueuePolicy'
    Properties:
      PolicyDocument:
        Statement:
          - Action: 'sqs:*'
            Condition:
              Bool:
                'aws:SecureTransport': 'false'
            Effect: Deny
            Principal:
              AWS: '*'
            Resource:
              'Fn::GetAtt':
                - AcceptedMessages
                - Arn
        Version: '2012-10-17'
      Queues: []

